In [19]:
import os
import re
import time
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta


# ігнорування попередження SettingWithCopyWarning
pd.set_option('mode.chained_assignment', None)  


In [3]:
# конвертування строчки дати в формат дати 
d, m, y = [int(x) for x in df.at[0, "Date"].split("/")]
a = date(y, m, d)
a
# date(2023, 5, 9).isoweekday() == 2  # первірка дати на вівторок

NameError: name 'df' is not defined

In [61]:
list_columns =   [
        "Date",
        "HomeTeam",
        "AwayTeam",
        "rez_h_full",
        "rez_a_full",
        "rez_h_first",
        "rez_a_first",
        "h_first",
        "h_full",
        "a_first",
        "a_full",
    ] 
# funcs

def get_team_results(
                    h_team_name: str,
                    a_team_name: str,
                    matches_before: pd.DataFrame,
                    ) -> tuple[int, int, int, int] :
    """
    отримання результатів команд, останнього матчу
    повертає список з 4 чисел, різниці забитих мячів
      - дом. команди в першому таймі,
      - дом. команди в повному матчі,
      - гост. команди в першому таймі,
      - гост. команди в повному матчі,
    """
    h_team_match = matches_before[                            
                        (matches_before["HomeTeam"] == h_team_name) | 
                        (matches_before["AwayTeam"] == h_team_name)
                    ].tail(1) 
    
    a_team_match = matches_before[
                        (matches_before["HomeTeam"] == a_team_name) |
                        (matches_before["AwayTeam"] == a_team_name)
                    ].tail(1) 

    h_first = (
                    h_team_match.at[h_team_match.index[0], "HTHG"] - 
                    h_team_match.at[h_team_match.index[0], "HTAG"]
                )


    h_full = (
                    h_team_match.at[h_team_match.index[0], "FTHG"] - 
                    h_team_match.at[h_team_match.index[0], "FTAG"]
                )

    h_first = h_first if h_team_name in h_team_match["HomeTeam"].values else -h_first
    h_full = h_full if h_team_name in h_team_match["HomeTeam"].values else -h_full


    a_first = (
                    a_team_match.at[a_team_match.index[0], "HTHG"] - 
                    a_team_match.at[a_team_match.index[0], "HTAG"]
                )

    a_full = (
                    a_team_match.at[a_team_match.index[0], "FTHG"] - 
                    a_team_match.at[a_team_match.index[0], "FTAG"]
                )
    a_first = a_first if a_team_name in a_team_match["HomeTeam"].values else -a_first
    a_full = a_full if a_team_name in a_team_match["HomeTeam"].values else -a_full
    
    return (
        h_first,
        h_full,
        a_first,
        a_full,
    )


def get_rez_data_df(df_data):
    rez_data = []
    for ind in df_data.index:
        current_match = df_data.loc[ind]
        matches_before = df_data.loc[:ind-1]
        h_team_name = current_match["HomeTeam"]
        a_team_name = current_match["AwayTeam"]
        if ( 
            (h_team_name in matches_before["HomeTeam"].values or
                     h_team_name in matches_before["AwayTeam"].values) and 
            (a_team_name in matches_before["HomeTeam"].values or
                     a_team_name in matches_before["AwayTeam"].values)  
        ):
            h_first, h_full, a_first, a_full = get_team_results(
                                                         h_team_name,
                                                         a_team_name,
                                                        matches_before
                                                        )
            """
            print(ind, h_team_name, a_team_name)
            print()
            print(matches_before.tail(15))
            print()

            print(h_full, h_first)

            print()
            print(a_full, a_first)
            """

            data_row = [
                *current_match.values,
                h_first,
                h_full,
                a_first,
                a_full
            ]
            rez_data.append(data_row)
    return rez_data


def get_clean_data_df(df):
    df.loc[:, "Date"] =  [date.fromisoformat("-".join(x.split("/")[::-1]))  for x in df.loc[:, "Date"].values]

    df_data = df.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HTHG", "HTAG"]]

    clean_data = pd.DataFrame(
                            data=get_rez_data_df(df_data),
                            columns=list_columns
                )
    return clean_data

In [58]:
def get_total_work(clean_data, tuesdays):

    rez_dict = {
        "win" : 0,
        "lose" : 0
    }
    
    for tuesday in tuesdays:

        work_df = clean_data[
            (clean_data["Date"] >= tuesday - timedelta(days=14)) &
            (clean_data["Date"] < tuesday) 
        ] 


        target_df = clean_data[
            (clean_data["Date"] >= tuesday)  &
            (clean_data["Date"] < tuesday + timedelta(days=7)) 
        ]

        # минулі результати
        work_df["index"] = work_df.index
        work_df_np = work_df[[
                                "h_first", "h_full",
                                "a_first", "a_full",
                                "rez_h_full", "rez_a_full", "rez_h_first", "rez_a_first",
                                "index"
                    ]].to_numpy()


        # майбутні результати
        target_df["index"] = target_df.index
        target_df_np = target_df[[
                                "h_first", "h_full",
                                "a_first", "a_full",
                                "rez_h_full", "rez_a_full", "rez_h_first", "rez_a_first",
                                "index"
                    ]].to_numpy()

        filtred_work_df_np = work_df_np[
            work_df_np[:, 6] == work_df_np[:, 7]
#             work_df_np[:,6] + work_df_np[:, 7] > 1.5
        ]
        if filtred_work_df_np.shape[0] == 0: continue

        for data in target_df_np:
            h_first = data[0]
            h_full = data[1]
            a_first = data[2]
            a_full = data[3]

            if not (
                    filtred_work_df_np[
#                         (filtred_work_df_np[:, 0] == h_first)
#                         &
                        (filtred_work_df_np[:, 1] == h_full)
#                         &
#                         (filtred_work_df_np[:, 2] == a_first)
                        &(filtred_work_df_np[:, 3] == a_full)
                    ].shape[0] >= 3        
            ): continue
            """
            print("="*25 )

            print(draw_work_df_np)
            print( )

            print(data )
            print( )
            """

            if data[6] == data[7]:
#             if data[6] + data[7] > 1.5:
                rez_dict["win"] += 1
            else:
                rez_dict["lose"] += 1

    return rez_dict    

In [1]:
total_rez_dict = {
    "win" : 0,
    "lose" : 0
}
for file_name in os.listdir("./old_data"):
    # df = pd.read_csv("./data/e0_21_22.csv")
    df = pd.read_csv(f"./data/{file_name}")
    # df = pd.read_csv("./data/sp1_21_22.csv")
    # df = pd.read_csv("./data/sp2_19_20.csv")
    # df = pd.read_csv("./data/n1_21_22.csv")


    clean_data = get_clean_data_df(df)

    start_day = clean_data["Date"].values[0]
    end_day = clean_data["Date"].values[-1]

    list_days = []
    while start_day <= end_day:
        list_days.append(start_day)
        start_day += timedelta(days=1)

    only_tuesdays = list(
                        filter(
                            lambda day: day.isoweekday() == 2,
                            list_days
                        )
                    )
    rez_dict = get_total_work(clean_data, only_tuesdays)
    win = rez_dict["win"]
    total_rez_dict["win"] += win
    
    lose = rez_dict["lose"]
    total_rez_dict["lose"] += lose
    
    proc = round(win/(win + lose), 3) if (win + lose) > 0 else 0
    print(
        file_name, win, lose, proc
    )
print(total_rez_dict, round(total_rez_dict["win"]/(total_rez_dict["win"]+total_rez_dict["lose"]), 3))
print(total_rez_dict["win"]*2.5 - total_rez_dict["win"] - total_rez_dict["lose"])

NameError: name 'os' is not defined

In [60]:
***h_first+h_full, при кількості співпадінь 1  - {'win': 398, 'lose': 497} 0.445 
***a_first+a_full,, при кількості співпадінь 1 - {'win': 394, 'lose': 518} 0.432


***h_first+h_full, при кількості співпадінь 2  - {'win': 154, 'lose': 157} 0.495
***a_first+a_full, при кількості співпадінь 2 -  {'win': 156, 'lose': 189} 0.452

h_first + a_first, при кількості співпадінь 2  {'win': 168, 'lose': 203} 0.453
====

h_first+h_full, при кількості співпадінь 3 - {'win': 121, 'lose': 163} 0.426 => 18.5   
a_first+a_full, при кількості співпадінь 3 - {'win': 124, 'lose': 140} 0.47 => 46
h_first + a_first, при кількості співпадінь 3 - {'win': 135, 'lose': 175} 0.435 => 27.5  
h_full + a_full, при кількості співпадінь 3  - {'win': 26, 'lose': 34} 0.433 => 5

SyntaxError: invalid syntax (3468536097.py, line 1)

In [12]:
df = pd.read_csv("./data/e0_21_22.csv")
df

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,13/08/2021,20:00,Brentford,Arsenal,2,0,H,1,0,...,1.62,0.50,1.75,2.05,1.81,2.13,2.05,2.17,1.80,2.09
1,E0,14/08/2021,12:30,Man United,Leeds,5,1,H,1,0,...,2.25,-1.00,2.05,1.75,2.17,1.77,2.19,1.93,2.10,1.79
2,E0,14/08/2021,15:00,Burnley,Brighton,1,2,A,1,0,...,1.62,0.25,1.79,2.15,1.81,2.14,1.82,2.19,1.79,2.12
3,E0,14/08/2021,15:00,Chelsea,Crystal Palace,3,0,H,2,0,...,1.94,-1.50,2.05,1.75,2.12,1.81,2.16,1.93,2.06,1.82
4,E0,14/08/2021,15:00,Everton,Southampton,3,1,H,0,1,...,1.67,-0.50,2.05,1.88,2.05,1.88,2.08,1.90,2.03,1.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,22/05/2022,16:00,Crystal Palace,Man United,1,0,H,1,0,...,2.04,0.25,1.68,2.15,1.74,2.23,1.88,2.25,1.74,2.16
376,E0,22/05/2022,16:00,Leicester,Southampton,4,1,H,0,0,...,2.63,-0.75,1.83,2.07,1.88,2.03,1.94,2.26,1.87,2.01
377,E0,22/05/2022,16:00,Liverpool,Wolves,3,1,H,1,1,...,3.28,-2.50,2.02,1.77,2.06,1.83,2.19,1.99,2.07,1.80
378,E0,22/05/2022,16:00,Man City,Aston Villa,3,2,H,0,1,...,3.36,-2.25,2.06,1.84,2.05,1.86,2.09,2.03,2.01,1.87
